# Refleksja w Javie

## Dostęp do informacji o typie, polach i metodach, dynamiczne ładowanie klas

<br/>

## dr inż. Aleksander Smywiński-Pohl

## apohllo@agh.edu.pl

## http://apohllo.pl/dydaktyka/programowanie-obiektowe

## konsultacje: środa 16:00 - 17:30, pokój 4.61

<img src="img/java_log.png" style="height: 700px"/>

In [1]:
class DummyIterator implements Iterator {
    private int index = 0;
    
    @Override
    public boolean hasNext(){
        return false;
    }
    
    @Override
    public Object next(){
        return null;
    }
}

In [ ]:
javac DummyIterator.java

In [ ]:
cat DummyIterator.class

```
����4
	indexI<init>()VCodeLineNumberTablehasNext()Znext()Ljava/lang/Object;
SourceFileDummyIterator.java
                             	
DummyIteratorjava/lang/Objectjava/util/Iterator      
&
*�*��



�
       
�
%        
```

In [ ]:
javap DummyIterator

In [ ]:
Compiled from "DummyIterator.java"
class DummyIterator implements java.util.Iterator {
  DummyIterator();
  public boolean hasNext();
  public java.lang.Object next();
}

# Informacje o typie

In [2]:
import static java.lang.System.out;

out.println(int.class);
out.println(Integer.class);
out.println("Ala ma kota".getClass());
out.println(new String[0].getClass());
out.println(new DummyIterator().getClass());

int
class java.lang.Integer
class java.lang.String
class [Ljava.lang.String;
class REPL.$JShell$12$DummyIterator


# Weryfikacja przynależności do typu

In [3]:
Object text = "To kot, a to Ala";
if(text.getClass().equals(String.class)){
   out.println("Obiekt jest stringiem");
}

Obiekt jest stringiem


In [4]:
Object iterator = new DummyIterator();
if(iterator.getClass().equals(Iterator.class)) {
    out.println("Obiekt jest iteratorem");
}

In [5]:
if(iterator instanceof Iterator) {
    out.println("Obiekt implementuje interfejs iterator");
}

Obiekt implementuje interfejs iterator


# Klasa nadrzędna

In [7]:
out.println(new DummyIterator().getClass());
out.println(new DummyIterator().getClass().getClass());
out.println(new DummyIterator().getClass().getClass().getClass());
out.println(new DummyIterator().getClass().getClass().getClass().getClass());
out.println(new DummyIterator().getClass().getClass().getClass().getClass().getSuperclass());
out.println(new DummyIterator().getClass().getClass().getClass().getClass().getSuperclass().getSuperclass());

class REPL.$JShell$12$DummyIterator
class java.lang.Class
class java.lang.Class
class java.lang.Class
class java.lang.Object
null


# Wspólna klasa nadrzędna

In [8]:
class Animal {}
class Mammal extends Animal {}
class Cat extends Mammal {}
class Whale extends Mammal {}
class Reptile extends Animal {}
class Snake extends Reptile {}

In [9]:
List<Class<?>> ancestors(Class<?> klass){
    var result = new LinkedList<Class<?>>();
    while(klass != Object.class){
        result.add(klass);
        klass = klass.getSuperclass();
    }
    result.add(Object.class);
    Collections.reverse(result);
    return result;
}

In [10]:
ancestors(Snake.class);

[class java.lang.Object, class REPL.$JShell$35$Animal, class REPL.$JShell$39$Reptile, class REPL.$JShell$40$Snake]

In [11]:
Class<?> commonAncestor(Class<?> aClass, Class<?> bClass){
    var aAncestors = ancestors(aClass);
    var bAncestors = ancestors(bClass);
    for(int i = 0; i < aAncestors.size() && i < bAncestors.size(); i++){
        if(aAncestors.get(i) != bAncestors.get(i)){
            return aAncestors.get(i-1);
        }
    }
    if(aAncestors.size() < bAncestors.size()){
        return aAncestors.get(aAncestors.size()-1);
    } else {
        return bAncestors.get(bAncestors.size()-1);
    }
}

In [13]:
out.println(commonAncestor(Cat.class, Snake.class));
out.println(commonAncestor(Cat.class, Whale.class));
out.println(commonAncestor(Cat.class, Animal.class));
out.println(commonAncestor(Animal.class, Cat.class));

class REPL.$JShell$35$Animal
class REPL.$JShell$36$Mammal
class REPL.$JShell$35$Animal
class REPL.$JShell$35$Animal


<img src="img/cat1.jpg" width="800"/>

# Dynamiczne ładowanie klas

In [14]:
interface IVehicle {
    void go();
}

In [15]:
class Car implements IVehicle {
    public void go(){
        out.println("Brum, brum");
    }
}
Car.class

class REPL.$JShell$51$Car

In [16]:
class Airplane implements IVehicle {
    public void go(){
        out.println("Bzzzzzz");
    }
}
Airplane.class

class REPL.$JShell$53$Airplane

In [17]:
class Rocket implements IVehicle {
    public void go(){
        out.println("BUUUUUUUUUUUM");
    }
}
Rocket.class

class REPL.$JShell$55$Rocket

In [18]:
class VehicleSystem {
    public static void main(String[] args){
        var vehicles = new LinkedList<IVehicle>();
        for(String vehicleName : args){
            try {
                var klass = (Class<IVehicle>) Class.forName(vehicleName);
                vehicles.add(klass.newInstance());
            } catch(ClassNotFoundException ex) {
                System.out.println("The class " + vehicleName +  
                                   " has not been found. ");
                return;
            } catch(InstantiationException | IllegalAccessException ex) {
                System.out.println("The class " + vehicleName +  
                                   " cannot be instantiated. ");
                return;
            }
            
        }
        for(IVehicle vehicle : vehicles) {
            vehicle.go();
        }
    }
}

In [21]:
String[] classes = {"REPL.$JShell$51$Car", "REPL.$JShell$55$Rocket", 
                    "REPL.$JShell$53$Airplane", "REPL.$JShell$51$Car"};
VehicleSystem.main(classes);

Brum, brum
BUUUUUUUUUUUM
Bzzzzzz
Brum, brum


<img src="img/cat2.jpg"/>

# Dostęp do składowych

* newInstance()
* getMethod(...)
* getMethods()
* getConstructor(...)
* getConstructors()
* getField(...)
* getFields()

In [22]:
void prettyPrint(Object[] elements){
    for(Object element : elements)
        out.println(element);
}

In [23]:
Class<?> klass = DummyIterator.class;

prettyPrint(klass.getMethods());

public boolean REPL.$JShell$12$DummyIterator.hasNext()
public java.lang.Object REPL.$JShell$12$DummyIterator.next()
public final void java.lang.Object.wait(long,int) throws java.lang.InterruptedException
public final void java.lang.Object.wait() throws java.lang.InterruptedException
public final native void java.lang.Object.wait(long) throws java.lang.InterruptedException
public boolean java.lang.Object.equals(java.lang.Object)
public java.lang.String java.lang.Object.toString()
public native int java.lang.Object.hashCode()
public final native java.lang.Class java.lang.Object.getClass()
public final native void java.lang.Object.notify()
public final native void java.lang.Object.notifyAll()
public default void java.util.Iterator.remove()
public default void java.util.Iterator.forEachRemaining(java.util.function.Consumer)


In [24]:
prettyPrint(klass.getConstructors());

public REPL.$JShell$12$DummyIterator()


In [25]:
prettyPrint(klass.getFields());

# Dynamiczne tworzenie instancji

In [26]:
import java.lang.reflect.*;

Class<?> klass = DummyIterator.class;
Constructor constructor = klass.getConstructor(new Class<?>[0]);
Iterator dummyIterator = (Iterator) constructor.newInstance();

System.out.println(dummyIterator);

REPL.$JShell$12$DummyIterator@68c989e0


In [27]:
class DummyIterator implements Iterator {
    private int index = 0;
    
    public DummyIterator(){
    }
    
    public DummyIterator(int index){
        this.index = index;
    }

    @Override
    public boolean hasNext(){
        return false;
    }

    @Override
    public Object next(){
        return null;
    }
    
    public String toString(){
        return "DummyIterator[" + this.index + "]";
    }
}

In [28]:
Class<?> klass = DummyIterator.class;

Constructor constructor = klass.getConstructor(new Class<?>[]{int.class});
Iterator dummyIterator = (Iterator) constructor.newInstance(1);

System.out.println(dummyIterator);

DummyIterator[1]


# Dynamiczne wywołanie metody

In [30]:
Method hasNextMethod = klass.getMethod("hasNext", new Class<?>[0]);
System.out.println(hasNextMethod.getName());
prettyPrint(hasNextMethod.getParameterTypes());

hasNext


In [31]:
Constructor constructor = klass.getConstructor(new Class<?>[]{int.class});
Iterator dummyIterator = (Iterator) constructor.newInstance(1);

Method hasNextMethod = klass.getMethod("hasNext", new Class<?>[0]);
hasNextMethod.invoke(dummyIterator);

false

# Dynamiczny dostęp do pól

In [32]:
Field indexField = klass.getField("index");
indexField.getName();
indexField.getModifiers();

EvalException: index

In [33]:
Field indexField = klass.getDeclaredField("index");
System.out.println(indexField.getName());
System.out.println(indexField.getModifiers());

index
2


In [34]:
Constructor constructor = klass.getConstructor(new Class<?>[]{int.class});
Iterator dummyIterator = (Iterator) constructor.newInstance(1);

Field indexField = klass.getDeclaredField("index");
indexField.get(dummyIterator);

EvalException: class REPL.$JShell$81 cannot access a member of class REPL.$JShell$12B$DummyIterator with modifiers "private"

In [35]:
indexField.setAccessible(true);
System.out.println(indexField.get(dummyIterator));
indexField.set(dummyIterator,2);
System.out.println(indexField.get(dummyIterator));

1
2


<img src="img/mouth.jpg" width="800"/>

# Dostęp do wszystkich składowych prywatnych

* getDeclaredMethod(...)
* getDeclaredMethods()
* getDeclaredConstructor(...)
* getDeclaredConstructors()
* getDeclaredField(...)
* getDeclaredFields()

Uwaga: składowe dziedziczone nie są zwracane

# Dynamiczne proxy

In [36]:
import java.lang.reflect.*;

class MethodReplayer implements InvocationHandler {
    private List<String> list = new LinkedList<>();
    private List<Method> invokedMethods = new LinkedList<>();
    private List<Object[]> invokedArguments = new LinkedList<>();

    public Object invoke(Object proxy, Method method, Object[] args) 
            throws Throwable{
        if(!method.getName().equals("toString")){
            invokedMethods.add(method);
            invokedArguments.add(args);
        }
        return method.invoke(list, args);
    }

    public void replay() throws Throwable {
        for(int i = 0; i < invokedMethods.size(); i++){
            invokedMethods.get(i).invoke(list, invokedArguments.get(i));
        }
    }
}

In [37]:
import static java.lang.reflect.Proxy.newProxyInstance;

MethodReplayer replayer = new MethodReplayer();
List<String> list = (List<String>) newProxyInstance(List.class.getClassLoader(),
        new Class[] { List.class },
        replayer);

list.add("dog");
list.add("cat");

System.out.println(list);


replayer.replay();
System.out.println(list);

[dog, cat]
[dog, cat, dog, cat]


<img src="img/cat3.jpg"/>

# Uchwyty do metod

* w API refleksji sygnatura argumentów metody jest wskazywana przez `Class[]`
* w nowym API mamy nowy typ `MethodType`:
  * brak nazwy metody
  * brak typu adresata
  * tylko typy argumentów oraz typ zwracanej wartości

In [38]:
import java.lang.invoke.*;
import java.lang.invoke.MethodHandles.Lookup;

// np. String toString()
MethodType toStringType = MethodType.methodType(String.class);

// np. Integer parseInt(String str)
MethodType fromStringToInt = MethodType.methodType(Integer.class, String.class);

In [39]:
class CallMeMaybe1 {
    public String toString(){
        return "call me maybe";
    }
   
    private String privateMethod(){
        return "Private string";
    }
    
    public Lookup getLookup(){
        return MethodHandles.lookup();
    }
}

In [40]:
CallMeMaybe1 maybe = new CallMeMaybe1();
Lookup maybeLookup = maybe.getLookup();

MethodHandle maybeHandle = maybeLookup.findVirtual(CallMeMaybe1.class, "toString", toStringType);
maybeHandle.invoke(maybe);

call me maybe

In [41]:
maybeHandle = maybeLookup.findSpecial(CallMeMaybe1.class, "privateMethod", toStringType, CallMeMaybe1.class);
maybeHandle.invoke(maybe);

Private string

In [42]:
maybeHandle = MethodHandles.lookup().findSpecial(CallMeMaybe1.class, "privateMethod", toStringType, CallMeMaybe1.class);
maybeHandle.invoke(maybe);

EvalException: no private access for invokespecial: class REPL.$JShell$99$CallMeMaybe1, from class REPL.$JShell$106 (unnamed module @6bcf54bf)

![Pytania? ](img/question.jpg)